In [ ]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import numpy as np
import plotly.express as px
from datetime import datetime
import calendar
def my_formatter(x):
    if x < 0:
        return '({:,.0f})'.format(-x)
    return '{:,.0f}'.format(x)

pd.options.display.float_format = my_formatter

## 整理Weekly report裡面2020年數據的BU

In [ ]:
# df = pd.read_excel('/Users/kai/OneDrive/INPAQ/業績總表/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

In [ ]:
# df['BU'].unique()

In [ ]:
# df['BU'] = df.apply(lambda x: x['BG'] if pd.isna(x['BU']) else x['BU'], axis=1)

In [ ]:
# df['BU'].unique()

In [ ]:
# filtered.groupby(['預交年份', '預交月份', 'BU', 'Group'])[['本國幣別NTD']].sum().reset_index()

## Susan需要的業績更新

In [ ]:
df = pd.read_excel('/Users/kai/OneDrive/INPAQ/業績總表/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

In [ ]:
budget = pd.read_excel('/Users/kai/OneDrive/INPAQ/業績總表/Weekly report_v0.1.xlsx', sheet_name='2021預算總表', usecols='a:j')

In [ ]:
filtered = df[(df['狀態'].str.contains('出')) & (df['預交年份'] == 2021) & (df['BU'] != 'Others')]

In [ ]:
filtered = filtered.groupby(['預交年份', '預交月份', 'BU'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
filtered['類別'] = '預計實績'

In [ ]:
budget = budget[(budget['預交年份'] == 2021)]

In [ ]:
filtered['BU'].unique()

In [ ]:
budget['BU'] = budget.apply(lambda x: x['BG'] if pd.isna(x['Subcategory']) else 'RF1' if x['Subcategory'] == 'BU1' else 'RF2' if x['Subcategory'] == 'BU2' else x['Subcategory'], axis=1)

In [ ]:
budget = budget.groupby(['預交年份', '預交月份', 'BU'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
budget['類別'] = '預算'

In [ ]:
final = pd.concat([filtered, budget])

## 快速生成英文月份

In [ ]:
month_order = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
# month_order = ['January', 'February', 'March', 'April', 'May', 
#               'June', 'July', 'August', 'September', 'October', 
#               'November', 'December']

## 繼續操作

In [ ]:
final['預交月份'] = pd.Categorical(final['預交月份'], ordered=True, categories=month_order)

In [ ]:
final_table = pd.pivot_table(final, index=['BU', '類別'], columns='預交月份', values='本國幣別NTD', fill_value=0, aggfunc=np.sum)

In [ ]:
chart = final.groupby(['BU', '類別', '預交月份'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
chart['本國幣別NTD'] = chart['本國幣別NTD'].fillna(0)

In [ ]:
chart['Category'] = chart['BU'].map(lambda x: 'RF' if x in ['RF1', 'RF2'] else 'Components')


In [ ]:
chart

In [ ]:
chart_rf = chart[~(chart['Category'] == 'Components')]

In [ ]:
chart_com = chart[(chart['Category'] == 'Components')]

In [ ]:
plot_by_bu = px.histogram(chart_rf, x='預交月份', y='本國幣別NTD', color='類別', barmode='group', facet_col='BU',
             labels=dict(BU='廠別', 本國幣別NTD='Amount', 預交月份='月份'), title='2021年RF產品各月份實績/預算 by BU')

In [ ]:
plot_rf = px.histogram(chart_rf, x='預交月份', y='本國幣別NTD', color='類別', barmode='group', 
            labels=dict(本國幣別NTD='Amount', 預交月份='月份'), title='2021年RF產品各月份實績/預算')

In [ ]:
plot_com = px.histogram(chart_com, x='預交月份', y='本國幣別NTD', color='BU', facet_col='類別', 
            labels=dict(本國幣別NTD='Amount', 預交月份='月份'), title='2021年元件產品各月份實績/預算')

In [ ]:
final_table

In [ ]:
today = datetime.today().strftime('%Y%m%d')

In [ ]:
with open('/Users/kai/OneDrive/INPAQ/業績總表/報告用圖表/' + today + '_Summary.html', 'a') as f:
    f.write(final_table.to_html())
    f.write(plot_rf.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(plot_by_bu.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(plot_com.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()